# Clustering Crypto

In [90]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import urllib, json, urllib.request

%matplotlib inline

### Fetching Cryptocurrency Data

In [91]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
response = urllib.request.urlopen(url)
data = json.loads(response.read())
coin_data = data['Data']

# Create a DataFrame 
coin_df = pd.DataFrame(coin_data)
coin_df = coin_df.transpose()
coin_df.head(1)

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,SortOrder,Sponsored,Taxonomy,Rating,IsTrading,TotalCoinsMined,BlockNumber,NetHashesPerSecond,BlockReward,BlockTime
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,34,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True,42,204348,0,0,0


In [92]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
###Create a DataFrame

### Data Preprocessing

In [93]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
coin_name = coin_df['CoinName']
coin_algo = coin_df['Algorithm']
coin_trade = coin_df['IsTrading']
coin_proof = coin_df['ProofType']
coin_mined = coin_df['TotalCoinsMined']
coin_supply = coin_df['TotalCoinSupply']
coin_df = pd.DataFrame(data=[coin_name, coin_algo, coin_trade,
                            coin_proof, coin_mined, coin_supply])
coin_df = coin_df.transpose()
coin_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.97105e+06,532000000
433,433 Token,N/A,False,N/A,1.12518e+08,1000000000


In [94]:
coin_df.shape

(5628, 6)

In [95]:
# Keep only cryptocurrencies that are trading
coin_df = coin_df[coin_df['IsTrading'] == True]

In [96]:
coin_df.shape

(4435, 6)

In [97]:
# Keep only cryptocurrencies with a working algorithm
coin_df = coin_df[coin_df['Algorithm'] != 'N/A']

In [98]:
coin_df.shape

(1464, 6)

In [99]:
# Remove the "IsTrading" column
coin_df.drop(columns='IsTrading',inplace=True)
coin_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.97105e+06,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0,0


In [100]:
# Remove rows with at least 1 null value
coin_df = coin_df[coin_df['TotalCoinSupply'] != '0']

# Remove rows with cryptocurrencies having no coins mined
coin_df = coin_df[coin_df['TotalCoinsMined'] > 0]

# Drop rows where there are 'N/A' text values
coin_df = coin_df[coin_df.TotalCoinSupply != 'NaN']
coin_df = coin_df[coin_df.TotalCoinSupply != 'N/A']

# Drop rows with NaN/Null/Na values
coin_df.dropna(axis=0, inplace=True)

print(coin_df.shape)
coin_df.head(3)

(526, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.97105e+06,532000000
1337,EliteCoin,X13,PoW/PoS,2.95049e+10,314159265359


In [101]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = coin_df['CoinName'].to_frame()
coin_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
NXT,Nxt
ZEC,ZCash


In [102]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
coin_df.drop(columns='CoinName',inplace=True)

In [103]:
coin_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.97105e+06,532000000
1337,X13,PoW/PoS,2.95049e+10,314159265359
NXT,PoS,PoS/LPoS,1000000000,1000000000
ZEC,Equihash,PoW,1.03684e+07,21000000


In [104]:
# Create dummy variables for text features
X = pd.get_dummies(data=coin_df, columns=['Algorithm','ProofType'])

In [105]:
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BEP2 BNB,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake,Algorithm_Blake2S,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,42,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.97105e+06,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.95049e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
NXT,1000000000,1000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,1.03684e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
STEEM,99692126,99692126,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTS,3600570502,3600570502,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTC,18530225,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WAVES,100000000,100000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Reducing Dimensions Using PCA

In [106]:
# Use StandardScaler to standardize all the data of the X DataFrame.
X_scaled = StandardScaler().fit_transform(X)

In [107]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
pca.fit_transform(X_scaled)

array([[-2.87935970e-01,  3.14032924e-01,  1.78484687e-02],
       [-2.82700155e-01,  3.13776831e-01,  1.78235377e-02],
       [ 3.25381421e+00,  8.59932936e-01,  1.02744290e-01],
       ...,
       [-2.17750562e-01, -9.24119877e-01, -7.44497710e-03],
       [-4.37206604e-01,  8.61249571e+00,  1.28633649e+01],
       [ 1.19360688e+00,  1.24104367e+00,  6.90424354e-02]])

In [108]:
pcs_df = pd.DataFrame(data=pca.fit_transform(X_scaled), columns=["PC 1","PC 2","PC 3"], index=dumb_coin_df.index)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.278849,0.209161,0.024716
404,-0.273602,0.208936,0.024667
1337,3.259482,0.839102,0.016002
NXT,-0.348596,1.175172,-0.143439
ZEC,-0.284933,-1.669474,-0.100308


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [109]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=42)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [118]:
# Predicting clusters with k=6

# Initialize the K-Means model
model = KMeans(n_clusters=6, random_state=42)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
pcs_df["Class"] = model.labels_

# Create a new DataFrame including predicted clusters and cryptocurrencies features


In [119]:
pcs_df

,PC 1,PC 2,PC 3,Class
42,-0.278849,0.209161,0.024716,0
404,-0.273602,0.208936,0.024667,0
1337,3.259482,0.839102,0.016002,4
NXT,-0.348596,1.175172,-0.143439,0
ZEC,-0.284933,-1.669474,-0.100308,5
...,...,...,...,...
STEEM,-0.307141,-0.236558,-0.047414,5
BTS,-0.178894,1.367248,0.141229,0
BTC,-0.221896,-0.895540,-0.067135,5
WAVES,-0.350483,8.377358,0.584748,0


In [120]:
#joining coin_df, pcs_df, coin_name_df
clust_df = coin_df.join(pcs_df)
clustered_df = clust_df.join(coin_name_df)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,42,42,-0.278849,0.209161,0.024716,0,42 Coin
404,Scrypt,PoW/PoS,1.97105e+06,532000000,-0.273602,0.208936,0.024667,0,404Coin
1337,X13,PoW/PoS,2.95049e+10,314159265359,3.259482,0.839102,0.016002,4,EliteCoin
NXT,PoS,PoS/LPoS,1000000000,1000000000,-0.348596,1.175172,-0.143439,0,Nxt
ZEC,Equihash,PoW,1.03684e+07,21000000,-0.284933,-1.669474,-0.100308,5,ZCash
...,...,...,...,...,...,...,...,...,...
STEEM,PoS,PoW,99692126,99692126,-0.307141,-0.236558,-0.047414,5,Steem
BTS,SHA-512,PoS,3600570502,3600570502,-0.178894,1.367248,0.141229,0,Bitshares
BTC,SHA-256,PoW,18530225,21000000,-0.221896,-0.895540,-0.067135,5,Bitcoin
WAVES,Leased POS,LPoS,100000000,100000000,-0.350483,8.377358,0.584748,0,Waves


### Visualizing Results

#### 3D-Scatter with Clusters

In [121]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(data_frame=clustered_df, hover_name="CoinName", hover_data=['Algorithm'])
fig.show()

#### Table of Tradable Cryptocurrencies

In [122]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [123]:
# Print the total number of tradable cryptocurrencies
print(len(clustered_df["CoinName"]))

526


#### Scatter Plot with Tradable Cryptocurrencies

In [131]:
# Scale data to create the scatter plot
scaled_coin_obj = StandardScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])

In [132]:
scaled_coin_df = pd.DataFrame(data=scaled_coin_obj, columns=["TotalCoinsMined","TotalCoinSupply"])

In [133]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
px.scatter(scaled_coin_df, x="TotalCoinsMined",y="TotalCoinSupply")